# Analyse COVID-19 : France, USA, Russie, Afrique du Sud

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Télécharger les données
url = 'https://covid.ourworldindata.org/data/owid-covid-data.csv'
df = pd.read_csv(url)

# Filtrage
pays = ['France', 'United States', 'Russia', 'South Africa']
df = df[df['location'].isin(pays)]

# Convertir les dates
df['date'] = pd.to_datetime(df['date'])

# Visualisation
plt.figure(figsize=(12, 6))
for country in pays:
    subset = df[df['location'] == country]
    subset = subset[['date', 'new_cases']].set_index('date')
    subset['new_cases'].rolling(7).mean().plot(label=country)

plt.title('Cas quotidiens (moyenne mobile 7 jours)')
plt.ylabel('Nouveaux cas')
plt.xlabel('Date')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()